In [1]:
# Start with loading all necessary libraries
import numpy as np
import pandas as pd
from os import path
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import os
import csv
import textract
import nltk
import joblib

import MeCab
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from konlpy.tag import *
from collections import Counter
from nltk.corpus import stopwords
import pyLDAvis.sklearn
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

%matplotlib inline

In [2]:
df = pd.read_csv('adoor_data/sns_feed.csv', encoding='UTF8')
df['created_at'] = pd.to_datetime(df['created_at'])

# df = df[df['author_id'] != 5]

start_date = pd.Timestamp(2019, 1, 28, 0)
end_date = pd.Timestamp(2019, 4, 1, 0)

mask = (df['created_at'] > start_date) & (df['created_at'] <= end_date)
df = df.loc[mask]

df.head()

,adoor,SNS,SNS 아이디,private,created_at,photo,content
0,2,facebook,김유리,False,2019-02-08,홍보,저와 팀원들이 영혼을 갈아넣은 서비스 베타버전이 드디어 나왔어요!!🤩\n밤잠을 줄여...
1,2,instagram,nnnyu_ri,False,2019-02-08,홍보,https://adoor.app @adoor.team\n저와 팀원들이 영혼을 갈아넣...
2,2,instagram,yulo_mon,True,2019-02-07,"가족, 아기, 강아지",카조꾸또 잇쇼니 토떼모 우레시깟딴데쓰
3,2,instagram,yulo_mon,True,2019-02-25,일상,🙋🏻‍♀️
4,2,instagram,yulo_mon,True,2019-03-02,나들이,필름캐머러 첫 롤 사진들1\n수라미가 준 로모그래피 심플유즈 기본 장착 컬러네거티브...


In [3]:
def getNVM_lemma(text):
    tokenizer = MeCab.Tagger()
    parsed = tokenizer.parse(text)
    word_tag = [w for w in parsed.split("\n")]
    pos = []
    tags = ['NNG','NNP','VV','VA', 'VX', 'VCP','VCN']
    for word_ in word_tag[:-2]:
        word = word_.split("\t")
        tag = word[1].split(",")
        if(len(word[0]) < 2) or ("게" in word[0]):
            continue
        if(tag[-1] != '*'):
            t = tag[-1].split('/')
            if(len(t[0]) > 1 and ('VV' in t[1] or 'VA' in t[1] or 'VX' in t[1])):
                pos.append(t[0])
        else:
            if(tag[0] in tags):
                pos.append(word[0])
    return pos

In [4]:
tf_vect = CountVectorizer(tokenizer=getNVM_lemma, min_df=2, max_df=6000, max_features=25000)
dtm = tf_vect.fit_transform(df['content'].values.astype('U'))

n_topics = 3

lda = LatentDirichletAllocation(n_components=n_topics, topic_word_prior=0.01, doc_topic_prior=0.001)
lda.fit(dtm)
saved_model = joblib.dump(dtm, 'LDA_IP.pkl')

In [5]:
names = tf_vect.get_feature_names()
topics_word = dict()
n_words = 10

for idx, topic in enumerate(lda.components_):
    vocab = []
    for i in topic.argsort()[:-(n_words-1):-1]:
        vocab.append((names[i], topic[i].round(2)))
    topics_word[idx+1] = [(names[i], topic[i].round(2)) for i in topic.argsort()[:-(n_words-1):-1]]
max_dict = dict()
for idx, vec in enumerate(lda.transform(dtm)):
    t = vec.argmax()
    if(t not in max_dict):
        max_dict[t] = (vec[t], idx)
    else:
        if(max_dict[t][0] < vec[t]):
            max_dict[t] = (vec[t], idx)
            
sorted_review = sorted(max_dict.items(), key = lambda x: x[0], reverse=False)

for key, value in sorted_review:
    print('주제 {}: {}'.format(key+1, topics_word[key+1]))
    print('[주제 {}의 대표 리뷰 :{}]\n{}\n\n'.format(key+1, value[0], df['content'].values.astype('U')[value[1]]))

주제 1: [('친구', 8.76), ('시간', 8.01), ('사람', 7.01), ('만들', 7.01), ('생각', 6.01), ('마음', 6.01), ('사용', 5.01), ('버리', 5.01)]
[주제 1의 대표 리뷰 :0.9999523843534986]
절 바지는 짱이다 태어나서 입어본 바지 중에 가장 얀입은듯 편한데 아주 따숩기까지 하다 거기다가 무척 힙해서 꼭 하나 사가고 싶었는데 불교용품점에서 58000원에 팔고 있어서 쓸쓸히 걸음을 돌리고 진수니와 3000원짜리 약과를 먹었다 스님들은 참 현명하다 평생을 이런 바지를 입는 길을 택하는 것은 아주 현명한 일이다
절밥은 무척 맛있고 속이 편하다 연잎밥은 연잎향이 그득하게 나고 브로콜리는 적당히 간이 되어있고 곤약볶음은 쫄깃하고 김치는 젓갈도 안들어간 비건이다 무척 맛있었지만 스님의 말씀을 어기고 많이 남기고 말았다 나는 아직 비움의 경지에 이르지 못했다
이불을 두 개 덮고 누운지 한시간 반인데 고등래퍼에 나온 랩들을 하나씩 다 읊어보기도 하고 그들의 첫인상을 되짚어보기도 했는데 잠이 들려고 할 때마다 옆에 언니들의 코고는 소리와 진수니의 아푼 기침소리가 겹쳐서 결국 정신이 맑아졌다 명상시간에는 너무 졸렸는데 이게 무슨 일이지
명상시간은 (공양시간 다음으로) 내가 제일 기대한 시간이었는데 몹시 실망스러웠다 일단 5분이었고 스님이 어려서부터 받은 상처를 생각해보세요 그것을 하나씩 들여다봐주세요 마음속 상처들을 다 블랙홀에 넣고10초를 세세요 심호흡을 하세요 요런 말들을 랩처럼 연속적으로 해서 생각할 시간이라고는 없었고 듣기만 하다 끝나버렸다
이 스님은 걸음도 몹시 빠르셔서 뛰지 않고서 내는 속도가 맞는지 의문스러울 정도이다 또한 여유로울 줄 알았던 스케줄은 이리 저리 뛰어다녀야 하고 자칫 무언가를 음미했다간 약속시간에 늦어버리는 빡빡한 일정이었다 의외로 절은 허슬라이프의 끝판왕이었던 것이다 엔타이 허슬을 하러 왔는데 너무 바빠버린다 스님의 걸음이 빠른 데에는 다 이유가 있는 것이다
스님의 일

In [6]:
visual = pyLDAvis.sklearn.prepare(lda_model=lda, dtm=dtm, vectorizer=tf_vect)
pyLDAvis.save_html(visual, 'LDA_Visualization2.html')
pyLDAvis.display(visual)